# Parallel calculations using Dask

From the [Dask documentation](https://docs.dask.org):

![](https://docs.dask.org/en/stable/_images/dask-overview.svg)

## 1. Create task graphs

### 1.1 Dask Delayed

In [ ]:
from dask.delayed import delayed

In [ ]:
@delayed
def add(x, y):
    print(f"{x} + {y}")
    return x + y

In [ ]:
a_p = add(1, 2)

In [ ]:
b_p = add(a_p, 3)

In [ ]:
c_p = add(a_p, b_p)

In [ ]:
import dask

In [ ]:
dask.visualize(c_p, rankdir="LR")

In [ ]:
c_p.compute()

### 1.2 Dask Arrays

In [ ]:
import dask.array as da

In [ ]:
x = da.random.random((2000, 1000), chunks=(500, 500))

In [ ]:
x

In [ ]:
y = da.dot(x, x.T)

In [ ]:
y

In [ ]:
z = y.mean()

In [ ]:
z

In [ ]:
dask.visualize(z)

In [ ]:
z.compute()

### 1.3 Xarray

From the [Xarray documentation](https://docs.xarray.dev): 
![](https://docs.xarray.dev/en/stable/_images/dataset-diagram.png)

In [ ]:
raster_path = '/project/stursdat/Data/RS-DAT/sentinel-2-l2a_AMS_2023-04/2023/4/30/S2B_31UFU_20230430_0_L2A/B02.tif'

In [ ]:
import rioxarray

In [ ]:
raster = rioxarray.open_rasterio(raster_path, chunks={"x": 2048, "y": 2048 }, lock=False)

In [ ]:
raster

In [ ]:
raster_max = raster.max()

In [ ]:
dask.visualize(raster_max)

In [ ]:
raster_max.compute()

## 2. Execute task graphs

### 2.1 Multi-threading/processing

In [ ]:
%%time
raster_max.compute(scheduler="threads", num_workers=2)

In [ ]:
%%time
raster_max.compute(scheduler="processes", num_workers=2)

### 2.2 Distributed scheduler

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.2.120:46409")
client

In [ ]:
raster_max.compute()